<a href="https://colab.research.google.com/github/KimJisanER/Automation_toyrent/blob/main/Binding_affinity_with_genetic_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q condacolab
import condacolab
condacolab.install()

import sys
sys.path

import condacolab
condacolab.check()

!mamba install -c conda-forge rdkit

✨🍰✨ Everything looks OK!
✨🍰✨ Everything looks OK!
/usr/local/lib/python3.10/site-packages/conda_package_streaming/package_streaming.py:19: UserWarning: zstandard could not be imported. Running without .conda support.
  warnings.warn("zstandard could not be imported. Running without .conda support.")
/usr/local/lib/python3.10/site-packages/conda_package_handling/api.py:29: UserWarning: Install zstandard Python bindings for .conda support
  _warnings.warn("Install zstandard Python bindings for .conda support")

                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔███

In [3]:
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from rdkit import Chem
from rdkit.Chem import PandasTools
from rdkit.Chem import AllChem
import rdkit
import rdkit.Chem as Chem
from rdkit.Chem.Draw import IPythonConsole
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import pickle
import random
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import r2_score
from tensorflow.keras import backend as K

In [4]:
def seed_everything(seed):
    # Python의 난수 생성기 시드 고정
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"

seed =42
seed_everything(seed)

In [5]:
with open(file='/content/drive/MyDrive/5_HTR/ligand_ki_0703.pickle', mode='rb') as f:
    df_Ki = pickle.load(f)

In [6]:
df_Ki

,chembl_id,pChEMBL_value_Ki,Fingerprint,target_accession
0,CHEMBL10270,9.70,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",Q64264
1,CHEMBL11388,7.26,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",Q64264
3,CHEMBL128329,4.50,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",Q64264
17,CHEMBL2021524,7.89,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",Q64264
18,CHEMBL2021538,8.00,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",Q64264
...,...,...,...,...
122085,CHEMBL4115424,6.64,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",Q923Y9
122086,CHEMBL4115473,7.54,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",Q923Y9
122087,CHEMBL4115499,7.97,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",Q923Y9
122088,CHEMBL4115571,8.16,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",Q923Y9


In [22]:
def embed_sequence(sequence):

    blosum62_matrix = {
        'A': [4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1, -1, -2, -1, 1, 0, -3, -2, 0],
        'R': [-1, 5, 0, -2, -3, 1, 0, -2, 0, -3, -2, 2, -1, -3, -2, -1, -1, -3, -2, -3],
        'N': [-2, 0, 6, 1, -3, 0, 0, 0, 1, -3, -3, 0, -2, -3, -2, 1, 0, -4, -2, -3],
        'D': [-2, -2, 1, 6, -3, 0, 2, -1, -1, -3, -4, -1, -3, -3, -1, 0, -1, -4, -3, -3],
        'C': [0, -3, -3, -3, 9, -3, -4, -3, -3, -1, -1, -3, -1, -2, -3, -1, -1, -2, -2, -1],
        'Q': [-1, 1, 0, 0, -3, 5, 2, -2, 0, -3, -2, 1, 0, -3, -1, 0, -1, -2, -1, -2],
        'E': [-1, 0, 0, 2, -4, 2, 5, -2, 0, -3, -3, 1, -2, -3, -1, 0, -1, -3, -2, -2],
        'G': [0, -2, 0, -1, -3, -2, -2, 6, -2, -4, -4, -2, -3, -3, -2, 0, -2, -2, -3, -3],
        'H': [-2, 0, 1, -1, -3, 0, 0, -2, 8, -3, -3, -1, -2, -1, -2, -1, -2, -2, 2, -3],
        'I': [-1, -3, -3, -3, -1, -3, -3, -4, -3, 4, 2, -3, 1, 0, -3, -2, -1, -3, -1, 3],
        'L': [-1, -2, -3, -4, -1, -2, -3, -4, -3, 2, 4, -2, 2, 0, -3, -2, -1, -2, -1, 1],
        'K': [-1, 2, 0, -1, -3, 1, 1, -2, -1, -3, -2, 5, -1, -3, -1, 0, -1, -3, -2, -2],
        'M': [-1, -1, -2, -3, -1, 0, -2, -3, -2, 1, 2, -1, 5, 0, -2, -1, -1, -1, -1, 1],
        'F': [-2, -3, -3, -3, -2, -3, -3, -3, -1, 0, 0, -3, 0, 6, -4, -2, -2, 1, 3, -1],
        'P': [-1, -2, -2, -1, -3, -1, -1, -2, -2, -3, -3, -1, -2, -4, 7, -1, -1, -4, -3, -2],
        'S': [1, -1, 1, 0, -1, 0, 0, 0, -1, -2, -2, 0, -1, -2, -1, 4, 1, -3, -2, -2],
        'T': [0, -1, 0, -1, -1, -1, -1, -2, -2, -1, -1, -1, -1, -2, -1, 1, 5, -2, -2, 0],
        'W': [-3, -3, -4, -4, -2, -2, -3, -2, -2, -3, -2, -3, -1, 1, -4, -3, -2, 11, 2, -3],
        'Y': [-2, -2, -2, -3, -2, -1, -2, -3, 2, -1, -1, -2, -1, 3, -3, -2, -2, 2, 7, -1],
        'V': [0, -3, -3, -3, -1, -2, -2, -3, -3, 3, 1, -2, 1, -1, -2, -2, 0, -3, -1, 4],
        'B': [-2, -1, 3, 4, -3, 0, 1, -1, 0, -3, -4, 0, -3, -3, -2, 0, -1, -4, -3, -3],
        'Z': [-1, 0, 0, 1, -3, 3, 4, -2, 0, -3, -3, 1, -1, -3, -1, 0, -1, -3, -2, -2],
        'X': [0, -1, -1, -1, -2, -1, -1, -1, -1, -1, -1, -1, -1, -1, -2, 0, 0, -2, -1, -1],
        '-': [-4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4],
        '*': [-4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4]
    }

    embedding = np.zeros((len(sequence), len(blosum62_matrix['A'])))
    for i, amino_acid in enumerate(sequence):
        if amino_acid not in blosum62_matrix:
            raise ValueError("Invalid amino acid: " + amino_acid)
        embedding[i] = blosum62_matrix[amino_acid]

    return embedding

def slicing(sequence, list):
    sequence_output = np.array(sequence[:20, list])
    return sequence_output

def process_seqs(lines):
    seqs = []
    headers = []
    header_positions = []

    for i, line in enumerate(lines):
        if line.startswith('>'):
            headers.append(line.strip().upper())
            seqs.append('')
            if '>SP' in line and 'HUMAN' in line and 'HTR' in line:
                header_positions.append(i)
        else:
            seqs[-1] += line.strip()

    seq_length = len(seqs[0])

    return headers, seqs, seq_length

def make_seqs_representation(headers, seqs, residue_numbers):
    data = []

    for header, seq in zip(headers, seqs):
        accession = header.split('|')[1]
        sequence = slicing(np.transpose(embed_sequence(seq) + 5)/16, residue_numbers)
        print(sequence.shape)
        data.append([accession, sequence])

    df = pd.DataFrame(data, columns=['target_accession', 'sequence'])
    return df

def convert_data(data):
  data_array = np.array([dt for dt in data])
  return data_array

In [13]:
def build_model(image_input_shape=(20, 12, 1), bit_input_shape=(2048, 2)):
    # 이미지 입력 경로
    image_input = keras.Input(shape=image_input_shape)
    x1_3x3 = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(image_input)
    x1_5x5 = layers.Conv2D(32, (1, 1), padding='same', activation='relu')(image_input)

    x1_3x3 = layers.MaxPooling2D((2, 2))(x1_3x3)
    x1_5x5 = layers.MaxPooling2D((2, 2))(x1_5x5)

    x1_3x3 = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x1_3x3)
    x1_5x5 = layers.Conv2D(64, (1, 1), padding='same', activation='relu')(x1_5x5)

    x1_3x3 = layers.MaxPooling2D((2, 2))(x1_3x3)
    x1_5x5 = layers.MaxPooling2D((2, 2))(x1_5x5)

    x1_3x3 = layers.Flatten()(x1_3x3)
    x1_5x5 = layers.Flatten()(x1_5x5)

    # 경로 병합
    x1 = layers.concatenate([x1_3x3, x1_5x5])
    x1 = layers.Dense(128, activation='relu')(x1)
    x1 = layers.BatchNormalization()(x1)

    # 비트 입력 경로
    bit_input = keras.Input(shape=bit_input_shape)
    x2 = layers.Flatten()(bit_input)

    # 경로 병합
    merged = layers.concatenate([x1, x2])
    merged = layers.Dense(256, activation='relu')(merged)
    merged = layers.BatchNormalization()(merged)
    merged = layers.Dropout(0.2)(merged)
    merged = layers.Dense(128, activation='relu')(merged)
    merged = layers.BatchNormalization()(merged)

    # 회귀 모델 출력 레이어 추가
    outputs = layers.Dense(1)(merged)

    # 모델 생성
    model = keras.Model(inputs=[image_input, bit_input], outputs=outputs)

    # 모델 컴파일
    model.compile(optimizer='adam', loss='mse')

    # # 모델 요약
    # model.summary()

    return model



In [14]:
fasta_file = '/content/drive/MyDrive/5_HTR/accessions_sequence_0606_muscle.fasta'

with open(fasta_file, 'r') as f:
    lines = f.readlines()

In [15]:
headers, seqs, seq_length = process_seqs(lines)

In [16]:
data_train, data_test = train_test_split(df_Ki, test_size=0.1, random_state=42)
data_train, data_val = train_test_split(data_train, test_size=0.1, random_state=42)

In [20]:
def score(residue_numbers):
 # Clear session and reinitialize the model
  K.clear_session()
  model = build_model()

  seq_df = make_seqs_representation(headers, seqs, residue_numbers)
  train = pd.merge(data_train, seq_df, on = 'target_accession')
  test = pd.merge(data_test, seq_df, on = 'target_accession')
  validation = pd.merge(data_val, seq_df, on = 'target_accession')

  X_train = train.drop(columns='pChEMBL_value_Ki')
  y_train = train['pChEMBL_value_Ki']

  X_test = test.drop(columns='pChEMBL_value_Ki')
  y_test = test['pChEMBL_value_Ki']

  X_val = validation.drop(columns='pChEMBL_value_Ki')
  y_val = validation['pChEMBL_value_Ki']

  train_inputs = [np.array(convert_data(X_train['sequence'])), np.array(convert_data(X_train['Fingerprint']))]
  val_inputs = [np.array(convert_data(X_val['sequence'])), np.array(convert_data(X_val['Fingerprint']))]
  test_inputs = [np.array(convert_data(X_test['sequence'])), np.array(convert_data(X_test['Fingerprint']))]

  checkpoint = ModelCheckpoint('/content/drive/MyDrive/HTR_only_model/best_model.h5', monitor='val_loss', save_best_only=True)
  history = model.fit(train_inputs, y_train, validation_data=(val_inputs, y_val), epochs=20, callbacks=[checkpoint])
  y_pred = model.predict(test_inputs)
  r2 = r2_score(y_test, y_pred)
  print('r2:', r2)
  K.clear_session()

  return r2

In [18]:
exp = [461, 466, 484, 1093, 360, 456, 1114, 1115, 1116, 285, 286, 300]
score(exp)

Epoch 1/20
2056/2056 [==============================] - ETA: 0s - loss: 3.6040

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


2056/2056 [==============================] - 29s 8ms/step - loss: 3.6040 - val_loss: 1.0889
Epoch 2/20
2056/2056 [==============================] - 13s 7ms/step - loss: 0.8516 - val_loss: 0.9659
Epoch 3/20
2056/2056 [==============================] - 14s 7ms/step - loss: 0.7663 - val_loss: 0.7964
Epoch 4/20
2056/2056 [==============================] - 14s 7ms/step - loss: 0.7049 - val_loss: 0.6810
Epoch 5/20
2056/2056 [==============================] - 13s 7ms/step - loss: 0.6547 - val_loss: 0.7044
Epoch 6/20
2056/2056 [==============================] - 13s 6ms/step - loss: 0.6234 - val_loss: 0.9231
Epoch 7/20
1394/2056 [===================>..........] - ETA: 4s - loss: 0.5943

KeyboardInterrupt: ignored

In [26]:
input_image = np.zeros((10, 10))
array_2d = np.random.rand(20, 12)

# 이미 할당된 좌표를 추적할 set을 생성합니다.
allocated_coordinates = set()

# 2D 배열의 12개의 요소를 중복되지 않는 랜덤한 좌표에 할당합니다.
for i in range(12):
    # 중복되지 않는 좌표를 선택합니다.
    while True:
        row_index = np.random.randint(0, 10)
        col_index = np.random.randint(0, 10)
        if (row_index, col_index) not in allocated_coordinates:
            break

    # 좌표를 할당하고 이미 할당된 좌표 set에 추가합니다.
    input_image[row_index, col_index] = array_2d[:, i]
    allocated_coordinates.add((row_index, col_index))

# 결과를 시각화합니다.
plt.imshow(input_image, cmap='gray')
plt.show()

ValueError: ignored

In [23]:
# Genetic algorithm parameters
population_size = 20
selection_size = 5
generations = 50
mutation_rate = 0.1

# Generate initial population
initial_population = [np.random.randint(0, 1450, size=12) for _ in range(population_size)]

# Main genetic algorithm loop
for generation in range(generations):
    print('Generation_No:', generation + 1)
    # Evaluate the fitness of each individual in the population
    fitness_scores = [score(residue_numbers) for residue_numbers in initial_population]

    # Select top individuals based on fitness scores
    selected_indices = np.argsort(fitness_scores)[-selection_size:]
    selected_population = [initial_population[i] for i in selected_indices]
    print(selected_population)

    # Crossover: create new individuals by combining sequences of selected individuals
    crossover_population = []

    for _ in range(population_size):
        random.shuffle(selected_population)
        parent1, parent2 = selected_population[:2]
        crossover_point = np.random.randint(1, 12)
        child = np.concatenate([parent1[:crossover_point], parent2[crossover_point:]])
        crossover_population.append(child)

    # Mutation: randomly modify some values in the population
    mutation_population = []
    for residue_numbers in crossover_population:
        if np.random.rand() < mutation_rate:
            mutation_point = np.random.randint(12)
            residue_numbers[mutation_point] = np.random.randint(0, 1450)
        mutation_population.append(residue_numbers)

    # Replace the initial population with the new population
    initial_population = mutation_population

# Get the best individual from the final population
best_residue_numbers = initial_population[np.argmax([score(residue_numbers) for residue_numbers in initial_population])]

print("Best residue_numbers:", best_residue_numbers)
print("Best score:", score(best_residue_numbers))

Generation_No: 1
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(20, 12)
(2

KeyboardInterrupt: ignored